### **1. Paramètres**

In [1]:
# Importation des bibliothèques nécessaires
import os 
import pandas as pd
import geopandas as gpd
import fiona

# Importation des fonctions
from functions_nettoyage import (nettoyer_gpkg, finaliser_gpkg)

### **1.1. Chargement des Données**

In [2]:
# Chargmenet du gpkg avec les données lidar
path_gpkg_brut = "../data_brut/vector/peupleraies_metriques.gpkg"
dep10_path = '../data_brut/vector/Peupliers_Dep10_32631.shp'

# Chemin du dossier de sortie
output_dir = "../data_final/vector/"

# Dictionnaire pour les chemins de sortie
gpkgs = {
    "parcelle": "../data_final/vector/peupleraies_lidar_parcelle.gpkg",
    "pixel": "../data_final/vector/peupleraies_lidar_pixel.gpkg"
}

In [3]:
# Chargement des vecteurs pour chaque département
dep47_brut = gpd.read_file(path_gpkg_brut, layer='clean_dset_peupliers_47_epsg2154')
dep73_brut = gpd.read_file(path_gpkg_brut, layer='parcelles_foret_chautagne_RGF93_avecCultivar_et_annees')
dep82_bb_brut = gpd.read_file(path_gpkg_brut, layer='GF_de_Borde_Basse_82')
dep82_sp_brut = gpd.read_file(path_gpkg_brut, layer='Carto_GFA_de_St_Pierre')
dep10_brut = gpd.read_file(dep10_path)

### **1.2. Configuration des colonnes et noms à conserver**

In [4]:
# Configuration des colonnes et noms à conserver
dep_configs = {
    "dep47": {
        "data": dep47_brut,
        "colonnes": ['Année_pl', 'Cultivars', 'PAI_GF_mean', 'VCI_mean',
                     'CC', 'MOCH', 'ENL', 'Z_mean', 'Densité', 'biomass_mean', 'lidar_date', 'geometry'],
        "noms": {'Année_pl': 'annee_plan', 'Cultivars': 'cultivar'}
    },
    "dep73": {
        "data": dep73_brut,
        "colonnes": ['Annee_plan', 'Cultivar1', 'PAI_GF_mean', 'VCI_mean',
                     'CC', 'MOCH', 'ENL', 'Z_mean', 'biomass_mean', 'lidar_date', 'geometry'],
        "noms": {'Annee_plan': 'annee_plan', 'Cultivar1': 'cultivar'}
    },
    "dep82_bb": {
        "data": dep82_bb_brut,
        "colonnes": ['Essence', 'Année_pla', 'PAI_GF_mean', 'VCI_mean',
                     'CC', 'MOCH', 'ENL', 'Z_mean', 'biomass_mean', 'lidar_date', 'geometry'],
        "noms": {'Année_pla': 'annee_plan', 'Essence': 'cultivar'}
    },
    "dep82_sp": {
        "data": dep82_sp_brut,
        "colonnes": ['Espece', 'Annee', 'PAI_GF_mean', 'VCI_mean',
                     'CC', 'MOCH', 'ENL', 'Z_mean', 'biomass_mean', 'lidar_date', 'geometry'],
        "noms": {'Annee': 'annee_plan', 'Espece': 'cultivar'}
    },
    "dep10": {
        "data": dep10_brut,
        "colonnes": ['d_essenc_1', 'd_essenc_2', 'Annee', 'geometry'],
        "noms": {'Annee': 'annee_plan', 'd_essenc_1': 'cultivar', 'd_essenc_2': 'd_essenc_2'}
    }
}

### **2. Nettoyage et tranformation des données**

In [5]:
# Boucle pour iterer sur les dictionaires en utilisant les fonctions crées
crs_cible = "EPSG:2154"  # Définir le CRS cible

# Création des répertoires de sortie si ils n'existent pas
os.makedirs(output_dir, exist_ok=True)

# Remover les anciens fichiers GeoPackage avant de créer les nouveaux
for gpkg_path in gpkgs.values():
    if os.path.exists(gpkg_path):
        os.remove(gpkg_path)
        print(f"Fichier supprimé: {gpkg_path}")


for echelle, gpkg_path in gpkgs.items():
    all_deps = []  # Pour stocker les GeoDataFrames nettoyés pour chaque échelle

    for couche_nom, config in dep_configs.items():
        # Reprojeter les données brutes vers le CRS cible
        data_reproj = config["data"].to_crs(crs_cible)

        # Nettoyer et finaliser les données
        dep_clean = nettoyer_gpkg(data_reproj, config["colonnes"], config["noms"], couche_nom, echelle)
        dep_clean = finaliser_gpkg(dep_clean, couche_nom, echelle)

        # Ajouter aux données globales
        all_deps.append(dep_clean)

        # Sauvegarder chaque couche individuellement
        dep_clean.to_file(gpkg_path, layer=f"{couche_nom}_{echelle}_clean", driver="GPKG")
    
    
    # Concaténer tous les départements pour l'échelle courante
    peupleraies_merged = gpd.GeoDataFrame(pd.concat(all_deps, ignore_index=True), crs=crs_cible)
    peupleraies_merged.to_file(gpkg_path, layer=f"peupleraies_merged_{echelle}", driver="GPKG")

    # Afficher les informations récapitulatives
    print(f"\nRésumé pour l'échelle: {echelle}")
    print(f"Nombre total de parcelles: {len(peupleraies_merged)}")
    unique_cultivars = peupleraies_merged['cultivar_n'].unique()
    print(f"Nombre de cultivars uniques: {len(unique_cultivars)}")
    print("Occurrences des cultivars triées par ordre décroissant:")
    print(peupleraies_merged['cultivar_n'].value_counts())

Fichier supprimé: ../data_final/vector/peupleraies_lidar_parcelle.gpkg
Fichier supprimé: ../data_final/vector/peupleraies_lidar_pixel.gpkg


C:\Users\caslu\AppData\Local\Temp\ipykernel_35540\2645599503.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  peupleraies_merged = gpd.GeoDataFrame(pd.concat(all_deps, ignore_index=True), crs=crs_cible)



Résumé pour l'échelle: parcelle
Nombre total de parcelles: 440
Nombre de cultivars uniques: 53
Occurrences des cultivars triées par ordre décroissant:
cultivar_n
Koster           66
I214             47
Raspalje         32
Polargo          30
Diva             26
I45/51           21
Tucano           21
Moncalvo         20
Soligo           18
Aleramo          13
Senna            12
Hoogvorst        11
Alcinde          11
San Martino      11
Rona             11
Dorskamp         10
Triplo            8
Vesten            8
Af8               6
Fritzy Pauley     6
Flevo             6
Albelo            4
Moleto            4
Af13              2
Lambro            2
Boelare           2
A4a               2
Delvignac         2
Degrosso          2
Dano              2
Imola             2
Faux Gaver        1
Trichobel         1
9 Clones          1
Taro              1
Turbo             1
Nikos             1
Palorn            1
Agora             1
Sprint            1
Ludo              1
Pannonia         

### **3. Vérification des couches finales**

In [6]:
# Vérification de la création des couches dans les fichiers GeoPackage
# Boucle pour chaque échelle et fichier GeoPackage
for echelle, gpkg_path in gpkgs.items():
    print(f"\nCouches disponibles dans le fichier GeoPackage : {gpkg_path}")
    # Liste des couches disponibles dans le GeoPackage
    for layer in fiona.listlayers(gpkg_path):
        print(layer)



Couches disponibles dans le fichier GeoPackage : ../data_final/vector/peupleraies_lidar_parcelle.gpkg
dep47_parcelle_clean
dep73_parcelle_clean
dep82_bb_parcelle_clean
dep82_sp_parcelle_clean
dep10_parcelle_clean
peupleraies_merged_parcelle

Couches disponibles dans le fichier GeoPackage : ../data_final/vector/peupleraies_lidar_pixel.gpkg
dep47_pixel_clean
dep73_pixel_clean
dep82_bb_pixel_clean
dep82_sp_pixel_clean
dep10_pixel_clean
peupleraies_merged_pixel


In [7]:
# Vérification des colonnes finales dans chaque fichier .gpkg
# Chemins des fichiers GeoPackage
path_gpkg_parcelle = '../data_final/vector/peupleraies_lidar_parcelle.gpkg'
path_gpkg_pixel = '../data_final/vector/peupleraies_lidar_pixel.gpkg'

# Chargement des couches vecteur
peupleraie_parcelle = gpd.read_file(path_gpkg_parcelle, layer='peupleraies_merged_parcelle')
peupleraie_pixel = gpd.read_file(path_gpkg_pixel, layer='peupleraies_merged_pixel')

# Affichage des colonnes pour vérification
print("Colonnes des peupleraies à l'échelle parcelle :", peupleraie_parcelle.columns)
print("Colonnes des peupleraies à l'échelle pixel :", peupleraie_pixel.columns)


Colonnes des peupleraies à l'échelle parcelle : Index(['unique_id', 'id_parc', 'annee_plan', 'cultivar_n', 'source',
       'PAI_GF_mean', 'VCI_mean', 'CC', 'MOCH', 'ENL', 'Z_mean', 'densite',
       'biomass_mean', 'lidar_date', 'geometry'],
      dtype='object')
Colonnes des peupleraies à l'échelle pixel : Index(['unique_id', 'id_parc', 'annee_plan', 'cultivar_n', 'source',
       'lidar_date', 'geometry'],
      dtype='object')
